In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 5.2 MB/s 
     |████████████████████████████████| 110 kB 44.6 MB/s 


In [ ]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sn
import random

import matplotlib.pyplot as plt

import contractions
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch import nn
import torch
import time


from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

In [ ]:
def get_detaset():
    train_df = pd.read_csv('/content/drive/MyDrive/IFT6390/kaggle2/train_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/IFT6390/kaggle2/test_data.csv')
    train_result_df = pd.read_csv('/content/drive/MyDrive/IFT6390/kaggle2/train_results.csv')
    return train_df, test_df, train_result_df

def treat_detaset(train_df, test_df, train_result_df):
    train_df= train_df.drop(columns=['id'])
    test_df = test_df.drop(columns=['id'])
    train_result_df = train_result_df.drop(columns=['id'])

    train_result_df.loc[train_result_df['target'] == 'negative'] = 0
    train_result_df.loc[train_result_df['target'] == 'neutral'] = 1
    train_result_df.loc[train_result_df['target'] == 'positive'] = 2
    return train_df, test_df, train_result_df

In [ ]:
train_df, test_df, train_result_df = get_detaset()
train_df, test_df, train_result_df = treat_detaset(train_df, test_df, train_result_df)

In [ ]:
#original text
print('positive',train_df[train_result_df.target == 2][0:3])
print('neutral',train_df[train_result_df.target == 1][0:3])
print('negative',train_df[train_result_df.target == 0][0:3])

In [ ]:
train_df.info()

In [ ]:
plt.figure(figsize=(6,5))
plt.title("Number of records of positive, negative, neutral sentiments")
plot = sn.countplot(x = 'target', data=train_result_df)
for p in plot.patches:
    plot.annotate(p.get_height(),(p.get_x()+0.1 ,p.get_height()+50))

In [ ]:
nltk.download("wordnet")
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def text_preprocessing(df):
    #1) Expand contractions in Text Processing
    df['reviews_text']=df['text'].apply(lambda x:contractions.fix(x, slang=True))
    #2) Lower Case
    df['reviews_text'] = df['reviews_text'].str.lower()
    #3) Remove punctuations
    df['reviews_text'] = df['reviews_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
    #4) Remove words containing digits
    df['reviews_text'] = df['reviews_text'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
    #5) Remove Stopwords
    #def remove_stopwords(text):
    #    return " ".join([word for word in str(text).split() if word not in stop_words])
    #df['reviews_text'] = df['reviews_text'].apply(lambda x: remove_stopwords(x))
    #6) Lemmatization
    def lemmatize_words(text):
        return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    df['reviews_text'] = df['reviews_text'].apply(lambda text: lemmatize_words(text))
    #7) Remove Extra Spaces
    df['reviews_text'] = df['reviews_text'].apply(lambda x: re.sub(' +', ' ', x))
    return df

In [ ]:
train_df_pre = text_preprocessing(train_df)
test_df_pre = text_preprocessing(test_df)

In [ ]:
train_df_pre.head(5)

In [ ]:
del train_df_pre['text']
del test_df_pre['text']

In [ ]:
train_df_pre.to_csv('train.txt', index=False, header = False)
test_df_pre.to_csv('validation.txt', index=False, header = False)

train_df_pre['target']=train_result_df['target']
train_df_pre['review_text']=train_df_pre['reviews_text']
del train_df_pre['reviews_text']

train_df_pre[:900000].to_csv('train_ro.txt', index=False, sep='\t', header = False)
train_df_pre[900000:].to_csv('valid_ro.txt', index=False, sep='\t', header = False)